In [2]:
import pandas as pd

# Load your main dataset
divergences_df = pd.read_csv('test-detected_divergences_for_multiple_stocks_with_next_bar_close.csv')
divergences_df['Second Date'] = pd.to_datetime(divergences_df['Second Date'])
divergences_df.sort_values(by=['Ticker', 'Second Date'], inplace=True)
divergences_df.reset_index(drop=True, inplace=True)

# Initialize columns for adjusted investments and position prices
divergences_df['Adjusted Investment'] = 1000  # Assuming an initial investment of 1000 units
divergences_df['Position Entrance Price'] = None
divergences_df['Position Closing Price'] = None
divergences_df['Final Investment Value'] = None  # New column for the final investment value after position closure

current_entrance_price = None
current_investment_value = 1000  # Starting investment value
fee_rate = 0.001  # Fee rate of 1%

# Initialize a counter for the number of times additional investment is added
additional_investment_count = 0

# Function to handle additional investment if value drops below zero
def handle_additional_investment(value):
    global additional_investment_count
    if value < 0:
        additional_investment_count += 1
        return 1000 + value  # Add additional $1000 if value drops below zero
    return value

# Iterate over the DataFrame rows to calculate the adjusted investment and record position prices
for index, row in divergences_df.iterrows():
    # Check if we are at the first row or if the ticker has changed
    if index == 0 or (divergences_df.loc[index, 'Ticker'] != divergences_df.loc[index - 1, 'Ticker']):
        # Handle calculation for the last row of the previous ticker
        if index != 0:
            last_row = divergences_df.loc[index - 1]
            final_close_price = last_row['Final Close']
            if last_row['Type'] == 'Bullish':
                gain_loss_percentage = (final_close_price - current_entrance_price) / current_entrance_price
            else:
                gain_loss_percentage = (current_entrance_price - final_close_price) / current_entrance_price
            gain_loss_percentage -= fee_rate * (1 if gain_loss_percentage > 0 else 1)
            current_investment_value *= (1 + gain_loss_percentage)
            current_investment_value = handle_additional_investment(current_investment_value)
            divergences_df.loc[index - 1, 'Final Investment Value'] = current_investment_value

        current_entrance_price = row['Next Bar Close']
        current_investment_value = 1000
        current_investment_value = handle_additional_investment(current_investment_value)
        divergences_df.loc[index, 'Position Entrance Price'] = current_entrance_price
        divergences_df.loc[index, 'Adjusted Investment'] = current_investment_value
    else:
        if divergences_df.loc[index, 'Type'] != divergences_df.loc[index - 1, 'Type']:
            previous_close_price = row['Next Bar Close']
            divergences_df.loc[index - 1, 'Position Closing Price'] = previous_close_price
            if row['Type'] == 'Bullish':
                gain_loss_percentage = (current_entrance_price - previous_close_price) / current_entrance_price
            else:
                gain_loss_percentage = (previous_close_price - current_entrance_price) / current_entrance_price
            gain_loss_percentage -= fee_rate * (1 if gain_loss_percentage > 0 else 1)
            current_investment_value *= (1 + gain_loss_percentage)

            current_investment_value = handle_additional_investment(current_investment_value)
            divergences_df.loc[index - 1, 'Final Investment Value'] = current_investment_value
            current_entrance_price = row['Next Bar Close']
            divergences_df.loc[index, 'Position Entrance Price'] = current_entrance_price
            divergences_df.loc[index, 'Adjusted Investment'] = current_investment_value
        else:
            divergences_df.loc[index, 'Position Entrance Price'] = current_entrance_price
            divergences_df.loc[index, 'Adjusted Investment'] = current_investment_value

if len(divergences_df) > 0:
    last_row = divergences_df.iloc[-1]
    final_close_price = last_row['Final Close']
    # Calculate the gain/loss percentage for the last position
    if last_row['Type'] == 'Bullish':
        gain_loss_percentage = (final_close_price - current_entrance_price) / current_entrance_price
    else:
        gain_loss_percentage = (current_entrance_price - final_close_price) / current_entrance_price

    # Adjust for the fee rate
    gain_loss_percentage -= fee_rate * (1 if gain_loss_percentage > 0 else 1)

    # Update the current investment value based on the gain/loss percentage
    current_investment_value *= (1 + gain_loss_percentage)

    # Handle additional investment if the value drops below zero for the last position
    current_investment_value = handle_additional_investment(current_investment_value)

    # Update the DataFrame with the final investment value for the last position
    divergences_df.iloc[-1, divergences_df.columns.get_loc('Final Investment Value')] = current_investment_value

    # Print the count of additional investments made throughout the process
    print(f"Additional investments were made {additional_investment_count} times.")

    # Save the adjusted DataFrame to a CSV file
    divergences_df.to_csv('test-divergences_with_adjusted_investments.csv', index=False)

    # Confirmation message
    print("Divergences with adjusted investments and the count of additional investments saved.")


/tmp/ipykernel_196702/1407443034.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2356.388232812675' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  divergences_df.loc[index, 'Adjusted Investment'] = current_investment_value


Additional investments were made 29 times.
Divergences with adjusted investments and the count of additional investments saved.
